# Additional Math and Stats Functions (Advanced)

This notebook contains **advanced (but not too much)** NumPy practice problems focused on:

- `amin` / `amax` (and `argmin` / `argmax`) with `axis`
- `mean` / `median` / `std` and robust statistics
- `sum` with axes and sanity checks
- rounding with `around`
- `histogram` for frequency distributions
- practical, vectorized implementations and validation

## Best practices used
- Reproducibility via random seeds
- Clear problem statements + expected behavior
- Vectorized solutions (avoid Python loops where reasonable)
- Validation with assertions / `np.testing`


In [1]:
import numpy as np

np.set_printoptions(precision=4, suppress=True)

def assert_close(a, b, *, rtol=1e-7, atol=1e-9, err_msg=""):
    np.testing.assert_allclose(a, b, rtol=rtol, atol=atol, err_msg=err_msg)

def assert_equal(a, b, *, err_msg=""):
    np.testing.assert_array_equal(a, b, err_msg=err_msg)


## Problem 1 — Column-wise min/max, and *where* they occur

Given the matrix `M`, compute:

1. `col_min`: the minimum of each column
2. `col_max`: the maximum of each column
3. `col_argmin`: the row index of the minimum in each column
4. `col_argmax`: the row index of the maximum in each column

Then verify:

- `M[col_argmin[j], j] == col_min[j]` for each column `j`
- `M[col_argmax[j], j] == col_max[j]` for each column `j`

**Tip:** use `np.amin/np.amax` and `np.argmin/np.argmax` with `axis=0`.

In [2]:
M = np.array([
    [ 5,  2,  9,  1],
    [ 7,  8, -3,  4],
    [ 0,  6,  2, 10],
    [ 3, -1, 11,  4],
])

col_min = np.amin(M, axis=0)
col_max = np.amax(M, axis=0)
col_argmin = np.argmin(M, axis=0)
col_argmax = np.argmax(M, axis=0)

# Validation (vectorized): pick entries M[row_index_per_col, col_index]
cols = np.arange(M.shape[1])
assert_equal(M[col_argmin, cols], col_min, err_msg="argmin indices do not match col_min")
assert_equal(M[col_argmax, cols], col_max, err_msg="argmax indices do not match col_max")

col_min, col_max, col_argmin, col_argmax

(array([ 0, -1, -3,  1]),
 array([ 7,  8, 11, 10]),
 array([2, 3, 1, 0]),
 array([1, 1, 3, 2]))

## Problem 2 — Normalize rows to percentages (safe division)

You are given a nonnegative matrix `X` representing counts per category (columns) for multiple groups (rows).

Compute `P`, where each row is converted to **percentages summing to 100**:

- `P[i, :] = X[i, :] / sum(X[i, :]) * 100`

Edge case: if a row sum is 0, return a row of zeros (avoid NaNs/Infs).

Finally, verify:
- rows with nonzero sum satisfy `P.sum(axis=1) == 100` (within floating tolerance)
- rows with zero sum satisfy `P.sum(axis=1) == 0`


In [3]:
X = np.array([
    [10,  0,  5],
    [ 0,  0,  0],
    [ 2,  3,  5],
    [ 1,  1,  1],
], dtype=float)

row_sums = np.sum(X, axis=1, keepdims=True)

# Safe division: only divide where row sum != 0
P = np.divide(X, row_sums, out=np.zeros_like(X), where=(row_sums != 0)) * 100

sums = P.sum(axis=1)
nonzero_rows = (row_sums[:, 0] != 0)
zero_rows = ~nonzero_rows

assert_close(sums[nonzero_rows], np.full(np.sum(nonzero_rows), 100.0), atol=1e-8)
assert_close(sums[zero_rows], np.zeros(np.sum(zero_rows)), atol=1e-8)

P

array([[66.6667,  0.    , 33.3333],
       [ 0.    ,  0.    ,  0.    ],
       [20.    , 30.    , 50.    ],
       [33.3333, 33.3333, 33.3333]])

## Problem 3 — Robust z-scores using median and MAD

Classic z-scores use mean and standard deviation, but can be sensitive to outliers.

Implement a **robust z-score** for a 1D array `x`:

- `median = np.median(x)`
- `mad = median(|x - median|)`  (MAD: median absolute deviation)
- `robust_z = 0.6745 * (x - median) / mad`

Edge case: if `mad == 0`, return zeros (avoid division by zero).

Compute robust z-scores for `x` below, and verify that the largest outlier has a large absolute robust z-score.


In [4]:
x = np.array([10, 11, 10, 12, 11, 10, 200], dtype=float)  # 200 is an outlier

med = np.median(x)
mad = np.median(np.abs(x - med))

if mad == 0:
    robust_z = np.zeros_like(x)
else:
    robust_z = 0.6745 * (x - med) / mad

# Sanity: outlier should stand out strongly
outlier_idx = np.argmax(np.abs(x - med))
assert np.abs(robust_z[outlier_idx]) > 10, "Outlier robust z-score should be large"

med, mad, robust_z

(np.float64(11.0),
 np.float64(1.0),
 array([ -0.6745,   0.    ,  -0.6745,   0.6745,   0.    ,  -0.6745,
        127.4805]))

## Problem 4 — Weighted average per column + compare to replication trick

Given observations (rows) and features (columns) in `A`, and nonnegative weights `w` for each row:

1. Compute the **weighted mean per column** using `np.average(A, axis=0, weights=w)`.
2. Validate the result by a *replication* approach:
   - scale to small integers and repeat rows accordingly, then compute the unweighted mean.

Use the provided data where the replication approach is feasible.


In [5]:
A = np.array([
    [1.0, 10.0, 100.0],
    [2.0, 20.0, 200.0],
    [3.0, 30.0, 300.0],
])
w = np.array([1, 2, 3], dtype=float)

wmean = np.average(A, axis=0, weights=w)

# Replication validation: repeat row i exactly w[i] times (w are small integers here)
repeated = np.repeat(A, repeats=w.astype(int), axis=0)
mean_rep = np.mean(repeated, axis=0)

assert_close(wmean, mean_rep, atol=1e-12)

wmean

array([  2.3333,  23.3333, 233.3333])

## Problem 5 — Histogram as a discrete probability model

Simulate 50,000 dice rolls (values 1–6). Then:

1. Build a histogram with bin edges `[1,2,3,4,5,6,7]` so each face is its own bin.
2. Convert counts to probabilities.
3. Compute the expected value `E[X]` from the histogram:
   - `E[X] = sum(face * prob(face))`

Verify `E[X]` is close to 3.5.


In [6]:
rng = np.random.default_rng(0)
rolls = rng.integers(1, 7, size=50_000)

bin_edges = np.arange(1, 8)  # [1,2,3,4,5,6,7]
counts, edges = np.histogram(rolls, bins=bin_edges)

probs = counts / counts.sum()
faces = edges[:-1]  # 1..6

EX = np.sum(faces * probs)
assert abs(EX - 3.5) < 0.05, f"Expected value too far from 3.5: got {EX}"

counts, probs, EX

(array([8340, 8277, 8456, 8405, 8263, 8259]),
 array([0.1668, 0.1655, 0.1691, 0.1681, 0.1653, 0.1652]),
 np.float64(3.4950200000000002))

## Problem 6 — Rounding strategy: bankers rounding vs always-up cents

You are given an array of prices with 3 decimal places. Compute two versions:

1. `bankers`: using `np.around(prices, 2)` (NumPy uses round-to-even behavior for ties)
2. `always_up`: round *up* to 2 decimals (ceiling at the cent):
   - `always_up = np.ceil(prices * 100) / 100`

Then compare both arrays.

**Goal:** understand that different rounding rules can change totals.


In [7]:
prices = np.array([1.005, 2.675, 3.335, 10.101, 10.100], dtype=float)

bankers = np.around(prices, 2)
always_up = np.ceil(prices * 100) / 100

# Compare totals
total_bankers = np.sum(bankers)
total_always_up = np.sum(always_up)

prices, bankers, always_up, total_bankers, total_always_up

(array([ 1.005,  2.675,  3.335, 10.101, 10.1  ]),
 array([ 1.  ,  2.68,  3.34, 10.1 , 10.1 ]),
 array([ 1.01,  2.68,  3.34, 10.11, 10.1 ]),
 np.float64(27.22),
 np.float64(27.240000000000002))

## Problem 7 — Multi-axis aggregation and consistency checks

Create a 3D array `T` with shape `(days, stores, products)`.
Interpretation: sales units.

Compute:

1. Total sales per day (sum over stores and products)
2. Total sales per store (sum over days and products)
3. Total sales per product (sum over days and stores)

Then check that all totals agree with the grand total:

- `grand_total == totals_per_day.sum() == totals_per_store.sum() == totals_per_product.sum()`


In [8]:
rng = np.random.default_rng(42)
T = rng.integers(0, 20, size=(7, 3, 5))  # 7 days, 3 stores, 5 products

totals_per_day = np.sum(T, axis=(1, 2))
totals_per_store = np.sum(T, axis=(0, 2))
totals_per_product = np.sum(T, axis=(0, 1))

grand_total = np.sum(T)

assert_equal(grand_total, totals_per_day.sum())
assert_equal(grand_total, totals_per_store.sum())
assert_equal(grand_total, totals_per_product.sum())

totals_per_day, totals_per_store, totals_per_product, grand_total

(array([153, 159, 133, 155, 149, 152, 150]),
 array([384, 317, 350]),
 array([221, 190, 200, 213, 227]),
 np.int64(1051))

## Problem 8 — Identify “most variable” column (std) and explain the pitfall

Given dataset `D` (rows are samples, columns are features):

1. Compute the standard deviation per column.
2. Find which column is most variable.
3. Now standardize the data (z-score per column) and recompute std.

**Key insight:** after z-scoring, every column should have std ~ 1 (depending on `ddof`).

Use `np.std(..., axis=0)` and z-score with broadcasting.


In [9]:
D = np.array([
    [ 1.0,  10.0, 100.0],
    [ 2.0,  12.0,  90.0],
    [ 3.0,   9.0, 110.0],
    [ 4.0,  11.0, 105.0],
    [ 5.0,  10.0,  95.0],
])

std0 = np.std(D, axis=0)  # population std
most_variable_col = int(np.argmax(std0))

mu = np.mean(D, axis=0)
sigma = np.std(D, axis=0)

# Safe: if any sigma == 0, avoid division by zero
Z = np.divide(D - mu, sigma, out=np.zeros_like(D), where=(sigma != 0))
stdZ = np.std(Z, axis=0)

# After z-score, std should be very close to 1 for non-constant columns
nonconst = (sigma != 0)
assert_close(stdZ[nonconst], np.ones(np.sum(nonconst)), atol=1e-12)

std0, most_variable_col, stdZ

(array([1.4142, 1.0198, 7.0711]), 2, array([1., 1., 1.]))